In [24]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [26]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [66]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Alagoas - Produção de Cimento (t)', axis=1)
data

,Ano,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Alagoas - Consumo de Cimento (t)
0,2008,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,0.655354,...,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016,446.335000
1,2009,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,0.658104,...,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837,471.072000
2,2010,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,0.660450,...,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447,583.772000
3,2011,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,0.662401,...,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959,742.881000
4,2012,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,0.663873,...,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248,844.864000
5,2013,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,0.664946,...,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582,791.375000
6,2014,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,0.666358,...,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262,818.645143
7,2015,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,0.666388,...,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007,676.300114
8,2016,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,0.678051,...,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435,538.586743
9,2017,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,0.677874,...,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980,362.791000


In [67]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.543035,0.601766,2.751323e+07,1.369962e+06,7.684387,2.402206e+07,0.749235,0.622573,0.655354,5.483335e+08,...,-7723.574973,2.868108,11.760983,19.541805,1004.903885,3228.342662,26.070097,20.787810,74.407297,3.079016
1,8.571316,0.606396,2.770792e+07,1.372464e+06,7.689682,2.403833e+07,0.751769,0.624024,0.658104,5.302764e+08,...,-8961.781709,2.305885,9.472585,20.736709,1004.528048,1611.970867,25.643630,21.162947,77.670858,2.815837
2,8.539868,0.610714,2.842678e+07,1.369925e+06,7.728365,2.421845e+07,0.753767,0.625020,0.660450,5.046425e+08,...,-9895.682385,1.958246,9.332485,19.768388,1004.613084,1481.706116,25.439288,21.037877,77.914366,2.782447
3,8.526852,0.614717,2.911152e+07,1.363070e+06,7.768204,2.438475e+07,0.755240,0.625568,0.662401,4.854622e+08,...,-9989.170714,2.079088,10.985689,18.126355,1004.876381,1381.865715,25.362543,20.922266,77.670210,2.776959
4,12.945391,0.618592,2.970505e+07,1.349554e+06,7.784639,2.449458e+07,0.756056,0.625434,0.663873,4.867661e+08,...,-9429.326328,2.183282,8.159014,17.184340,1005.022203,1424.662145,25.342566,20.944017,77.821942,2.740248
5,13.577910,0.621225,3.023816e+07,1.337956e+06,7.798417,2.459006e+07,0.756644,0.625304,0.664946,5.225848e+08,...,-7157.441850,2.403445,7.913561,17.252127,1005.229473,1469.493466,25.323659,21.009855,78.181963,2.678582
6,14.169213,0.625244,3.070999e+07,1.324714e+06,7.820611,2.467763e+07,0.756698,0.624996,0.666358,5.676413e+08,...,-3453.461327,2.748397,10.350442,16.874158,1005.478116,1490.005745,25.397446,21.064696,78.095570,2.657262
7,14.659464,0.627431,3.107611e+07,1.310803e+06,7.837546,2.473871e+07,0.756172,0.623329,0.666388,5.179392e+08,...,-1442.919387,2.501529,12.454398,16.443616,1005.522737,1495.198000,25.490999,21.080729,77.734091,2.687007
8,14.776623,0.643045,3.127251e+07,1.307005e+06,7.840584,2.474349e+07,0.767870,0.631369,0.678051,4.524144e+08,...,-3128.377228,2.073464,13.111908,16.926315,1005.453192,1487.897624,25.451016,21.102972,78.001380,2.705435
9,14.347557,0.643776,3.135879e+07,1.302182e+06,7.848958,2.475540e+07,0.767131,0.630858,0.677874,4.213689e+08,...,-5386.337108,2.177386,9.479812,17.228816,1005.412538,1488.339609,25.438238,21.210017,78.555457,2.646980


In [68]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
12    576.578000
13    547.622000
14           NaN
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [69]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.394011,-1.562212,-1.771772,1.267308,-1.824617,-1.828417,-1.530739,-1.310765,-1.525249,1.204752,...,-0.569731,2.026352,0.984445,1.186951,-0.595151,3.297433,1.414461,-1.989133,-2.084147,2.701872
1,-1.383296,-1.252696,-1.630985,1.357232,-1.734035,-1.767156,-1.104764,-0.842247,-1.175856,0.820657,...,-0.838980,0.085558,-0.051710,1.903618,-1.755575,-0.042015,0.116552,0.600096,0.287340,0.438290
2,-1.395211,-0.964094,-1.111174,1.266004,-1.072298,-1.088666,-0.769045,-0.520929,-0.877896,0.275392,...,-1.042057,-1.114490,-0.115145,1.322848,-1.493020,-0.311144,-0.505344,-0.263151,0.464287,0.151102
3,-1.400143,-0.696569,-0.616035,1.019651,-0.390788,-0.462241,-0.521550,-0.344017,-0.630022,-0.132596,...,-1.062386,-0.697344,0.633403,0.338007,-0.680070,-0.517416,-0.738908,-1.061109,0.286869,0.103900
4,0.274014,-0.437536,-0.186847,0.533959,-0.109633,-0.048522,-0.384411,-0.387101,-0.443003,-0.104860,...,-0.940648,-0.337667,-0.646476,-0.226984,-0.229833,-0.428998,-0.799707,-0.910979,0.397127,-0.211851
5,0.513671,-0.261567,0.198648,0.117149,0.126049,0.311122,-0.285597,-0.429212,-0.306719,0.657046,...,-0.446625,0.422337,-0.757614,-0.186328,0.410129,-0.336376,-0.857248,-0.456556,0.658738,-0.742231
6,0.737712,0.007045,0.539832,-0.358708,0.505729,0.641003,-0.276579,-0.528629,-0.127318,1.615453,...,0.358807,1.613108,0.345772,-0.413022,1.177832,-0.293997,-0.632685,-0.078042,0.595960,-0.925602
7,0.923465,0.153222,0.804578,-0.858614,0.795419,0.871076,-0.364842,-1.066695,-0.123527,0.558230,...,0.796000,0.760923,1.298413,-0.671248,1.315604,-0.283270,-0.347965,0.032622,0.333289,-0.669771
8,0.967856,1.196863,0.946595,-0.995086,0.847385,0.889099,1.600990,1.528559,1.358195,-0.835563,...,0.429496,-0.716758,1.596124,-0.381740,1.100879,-0.298352,-0.469649,0.186147,0.527516,-0.511267
9,0.805286,1.245745,1.008989,-1.168405,0.990653,0.933950,1.476783,1.363339,1.335691,-1.495939,...,-0.061498,-0.358019,-0.048438,-0.200310,0.975357,-0.297439,-0.508538,0.924979,0.930139,-1.014033


In [70]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     471.072000
1     583.772000
2     742.881000
3     844.864000
4     791.375000
5     818.645143
6     676.300114
7     538.586743
8     362.791000
9     340.278000
10    395.359000
11    581.479000
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [71]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.540735,1.153503,0.54759,-0.669467,0.504915,0.391992,0.441591,0.935129,0.989474,1.108405,...,0.964765,-1.482779,-2.260447,-1.676368,-0.895465,-0.283979,1.831602,0.665102,-1.743991,0.61839


In [72]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    576.578
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [95]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [96]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [97]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3043689624, 1076993307, 2957001173, 2006710543, 841097278, 462778655, 2074158358, 3609671121, 666631248, 3524487745, 1930995868, 1896799421, 13285143, 1819717945, 746012154, 2750012199, 4161197484, 828443865, 1248294379, 2148532938, 847273135, 3442687473, 688929262, 1350970240, 4132362975, 570653636, 4095466841, 1349871806, 2886615658, 4280032504, 846137551, 2103937557, 128786492, 2344510030, 2478269859, 2371935831, 1673183575, 4232824121, 3431513274, 560644020, 2418164437, 1415118966, 2645452179, 1657888050, 3590567097, 3157734108, 1755985142, 3664448240, 1484093159, 1976022109, 1948324107, 1162272126, 468199496, 3117193318, 289692708, 3868060356, 1733569998, 2277942648, 4230353621, 1203070206, 830422264, 3266974254, 4235859373, 1473835924, 58544734, 3184097010, 3556153041, 3175918514, 3367330360, 2889885547, 1853604973, 204546413, 2671359047, 3369469420, 330690334, 2760169650, 2284511470, 2751670632, 2556377221, 1650118662, 879707933, 2089014100, 2326573126, 1726156118, 2464276320, 

val_loss: 2022.3037109375


Step: 85 ___________________________________________
val_loss: 21138.17578125


Step: 86 ___________________________________________
val_loss: 26064.10546875


Step: 87 ___________________________________________
val_loss: 10690.5927734375


Step: 88 ___________________________________________
val_loss: 6593.541015625


Step: 89 ___________________________________________
val_loss: 1168.60986328125


Step: 90 ___________________________________________
val_loss: 13229.5419921875


Step: 91 ___________________________________________
val_loss: 7600.17041015625


Step: 92 ___________________________________________
val_loss: 23987.021484375


Step: 93 ___________________________________________
val_loss: 6173.64404296875


Step: 94 ___________________________________________
val_loss: 16981.275390625


Step: 95 ___________________________________________
val_loss: 21282.595703125


Step: 96 ___________________________________________
val_loss: 15593.00390625



In [98]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 478ms/step - loss: 384469.0000 - val_loss: 314389.4375
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 375181.0312 - val_loss: 518893.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 346933.4375 - val_loss: 474423.4688
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 366371.0938 - val_loss: 584346.5625
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 280915.9062 - val_loss: 658190.3125
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 281907.3438 - val_loss: 636270.5000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 272296.8125 - val_loss: 680449.5000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 250303.6406 - val_loss: 679350.9375
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 243974.9219 - val_loss: 695977.4375
Epoch 10/

Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 92230.3203 - val_loss: 207068.4688
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 91417.3516 - val_loss: 189826.2500
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 94296.2266 - val_loss: 163627.9531
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 89565.5938 - val_loss: 194272.5312
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 89042.4609 - val_loss: 177886.3906
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 91213.1172 - val_loss: 114029.9219
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 99074.0625 - val_loss: 178408.0156
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 90743.6875 - val_loss: 159387.8594
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 90219.7500 - val_loss: 166435.2656
Epoch 84/1

1/1 [==============================] - 0s 14ms/step - loss: 29189.1934 - val_loss: 88885.5938
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 28074.9473 - val_loss: 82321.2578
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 28113.5410 - val_loss: 77633.3516
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 25988.5879 - val_loss: 71384.2188
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 27172.7598 - val_loss: 69272.5078
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 22578.8066 - val_loss: 65410.2148
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 24723.1016 - val_loss: 58080.8828
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 23459.0254 - val_loss: 58295.9414
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 23866.4785 - val_loss: 56472.5000
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 17ms/step - loss: 11603.7881 - val_loss: 12973.0908
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 11807.8604 - val_loss: 12009.0713
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 11696.6045 - val_loss: 12024.5605
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 11696.9531 - val_loss: 12185.8809
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 11448.0029 - val_loss: 11801.7842
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 11368.5049 - val_loss: 12345.5098
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 11284.3252 - val_loss: 12302.7891
Epoch 230/10000
1/1 [==============================] - 0s 15ms/step - loss: 11184.2461 - val_loss: 11966.9629
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 11729.1211 - val_loss: 11763.9600
Epoch 232/10000
1/1 [=====

Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 15128.5547 - val_loss: 21802.4590
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 15010.8828 - val_loss: 15829.0459
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 14403.8174 - val_loss: 16755.5352
Epoch 301/10000
1/1 [==============================] - 0s 18ms/step - loss: 13090.2422 - val_loss: 10086.2510
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 13382.7188 - val_loss: 10939.3037
Epoch 303/10000
1/1 [==============================] - 0s 18ms/step - loss: 12570.0791 - val_loss: 12006.3223
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 11435.1826 - val_loss: 9819.8857
Epoch 305/10000
1/1 [==============================] - 0s 15ms/step - loss: 12003.4971 - val_loss: 11935.1221
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 11298.7666 - val_loss: 11758.7051
Epoch 307/1

...

1/1 [==============================] - 0s 14ms/step - loss: 10275.4424 - val_loss: 245.1487
Epoch 1202/10000
1/1 [==============================] - 0s 14ms/step - loss: 10274.8525 - val_loss: 244.7953
Epoch 1203/10000
1/1 [==============================] - 0s 14ms/step - loss: 10274.2656 - val_loss: 244.4344
Epoch 1204/10000
1/1 [==============================] - 0s 14ms/step - loss: 10273.6807 - val_loss: 244.0777
Epoch 1205/10000
1/1 [==============================] - 0s 14ms/step - loss: 10273.1006 - val_loss: 243.7174
Epoch 1206/10000
1/1 [==============================] - 0s 14ms/step - loss: 10272.5225 - val_loss: 243.3554
Epoch 1207/10000
1/1 [==============================] - 0s 14ms/step - loss: 10271.9482 - val_loss: 242.9937
Epoch 1208/10000
1/1 [==============================] - 0s 18ms/step - loss: 10271.3750 - val_loss: 242.6285
Epoch 1209/10000
1/1 [==============================] - 0s 14ms/step - loss: 10270.8066 - val_loss: 242.2617
Epoch 1210/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 10239.4482 - val_loss: 209.7123
Epoch 1277/10000
1/1 [==============================] - 0s 13ms/step - loss: 10239.0342 - val_loss: 209.3447
Epoch 1278/10000
1/1 [==============================] - 0s 14ms/step - loss: 10238.6221 - val_loss: 208.9722
Epoch 1279/10000
1/1 [==============================] - 0s 14ms/step - loss: 10238.2119 - val_loss: 208.6053
Epoch 1280/10000
1/1 [==============================] - 0s 14ms/step - loss: 10237.8037 - val_loss: 208.2335
Epoch 1281/10000
1/1 [==============================] - 0s 14ms/step - loss: 10237.3975 - val_loss: 207.8620
Epoch 1282/10000
1/1 [==============================] - 0s 14ms/step - loss: 10236.9922 - val_loss: 207.4873
Epoch 1283/10000
1/1 [==============================] - 0s 14ms/step - loss: 10236.5898 - val_loss: 207.1147
Epoch 1284/10000
1/1 [==============================] - 0s 14ms/step - loss: 10236.1885 - val_loss: 206.7372
Epoch 1285/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 13192.9531 - val_loss: 31948.9883
Epoch 1351/10000
1/1 [==============================] - 0s 14ms/step - loss: 13191.4170 - val_loss: 31916.2676
Epoch 1352/10000
1/1 [==============================] - 0s 14ms/step - loss: 13189.8994 - val_loss: 31883.7812
Epoch 1353/10000
1/1 [==============================] - 0s 14ms/step - loss: 13188.4033 - val_loss: 31851.5625
Epoch 1354/10000
1/1 [==============================] - 0s 13ms/step - loss: 13186.9268 - val_loss: 31819.5684
Epoch 1355/10000
1/1 [==============================] - 0s 14ms/step - loss: 13185.4697 - val_loss: 31787.8496
Epoch 1356/10000
1/1 [==============================] - 0s 14ms/step - loss: 13184.0342 - val_loss: 31756.3535
Epoch 1357/10000
1/1 [==============================] - 0s 14ms/step - loss: 13182.6172 - val_loss: 31725.0918
Epoch 1358/10000
1/1 [==============================] - 0s 14ms/step - loss: 13181.2188 - val_loss: 31694.1055
Epoch 1359/10000
1

Epoch 1424/10000
1/1 [==============================] - 0s 14ms/step - loss: 13118.1885 - val_loss: 30071.7383
Epoch 1425/10000
1/1 [==============================] - 0s 14ms/step - loss: 13117.5439 - val_loss: 30052.3613
Epoch 1426/10000
1/1 [==============================] - 0s 14ms/step - loss: 13116.9062 - val_loss: 30033.0762
Epoch 1427/10000
1/1 [==============================] - 0s 14ms/step - loss: 13116.2764 - val_loss: 30013.9336
Epoch 1428/10000
1/1 [==============================] - 0s 14ms/step - loss: 13115.6494 - val_loss: 29994.9141
Epoch 1429/10000
1/1 [==============================] - 0s 16ms/step - loss: 13115.0303 - val_loss: 29976.0273
Epoch 1430/10000
1/1 [==============================] - 0s 18ms/step - loss: 13114.4189 - val_loss: 29957.2422
Epoch 1431/10000
1/1 [==============================] - 0s 18ms/step - loss: 13113.8115 - val_loss: 29938.5996
Epoch 1432/10000
1/1 [==============================] - 0s 18ms/step - loss: 13113.2109 - val_loss: 29920.0684
E

Epoch 1498/10000
1/1 [==============================] - 0s 14ms/step - loss: 13083.7197 - val_loss: 28914.0195
Epoch 1499/10000
1/1 [==============================] - 0s 14ms/step - loss: 13083.3916 - val_loss: 28901.5254
Epoch 1500/10000
1/1 [==============================] - 0s 14ms/step - loss: 13083.0674 - val_loss: 28889.0859
Epoch 1501/10000
1/1 [==============================] - 0s 14ms/step - loss: 13082.7451 - val_loss: 28876.7617
Epoch 1502/10000
1/1 [==============================] - 0s 14ms/step - loss: 13082.4248 - val_loss: 28864.4414
Epoch 1503/10000
1/1 [==============================] - 0s 14ms/step - loss: 13082.1064 - val_loss: 28852.2285
Epoch 1504/10000
1/1 [==============================] - 0s 14ms/step - loss: 13081.7930 - val_loss: 28840.0684
Epoch 1505/10000
1/1 [==============================] - 0s 14ms/step - loss: 13081.4805 - val_loss: 28827.9727
Epoch 1506/10000
1/1 [==============================] - 0s 14ms/step - loss: 13081.1719 - val_loss: 28815.9531
E

In [99]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 37ms/step
[576.578](test_target) - [[581.4438]](prediction) = 4.8657866210937755


In [100]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [101]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.592143,-1.238768,-0.940263,-0.691783,-0.837853,-0.796510,-1.270151,-1.293364,-1.255860,1.148205,...,1.277766,1.308949,1.412346,-0.914375,1.380911,1.411042,1.341066,-1.336272,-1.411478,1.406725
1,1.408288,0.028544,-0.444707,1.414104,-0.567735,-0.613762,0.096525,0.151296,0.064802,0.140893,...,-0.114016,-0.190804,-0.643250,1.391499,-0.954679,-0.623537,-0.281746,1.069113,0.629592,-0.577490
2,-0.816144,1.210223,1.384970,-0.722321,1.405588,1.410272,1.173626,1.142068,1.191057,-1.289098,...,-1.163750,-1.118144,-0.769096,-0.477124,-0.426232,-0.787505,-1.059321,0.267160,0.781885,-0.829235


train_target:


,Alagoas - Consumo de Cimento (t)
0,471.072
1,583.772
2,742.881


test_input:


...

,Alagoas - Desemprego,Alagoas - IDH Educacao,Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - IDH Longevidade,Alagoas - IDH Renda,Alagoas - IDH,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.540735,1.153503,0.54759,-0.669467,0.504915,0.391992,0.441591,0.935129,0.989474,1.108405,...,0.964765,-1.482779,-2.260447,-1.676368,-0.895465,-0.283979,1.831602,0.665102,-1.743991,0.61839


test_target:


,Alagoas - Consumo de Cimento (t)
12,576.578


1/1 [==============================] - 0s 34ms/step
Error: 4.8657866210937755




[742.8809814453125,
 793.86962890625,
 777.3408813476562,
 711.07470703125,
 681.372314453125,
 538.5426635742188,
 393.5416564941406,
 367.07562255859375,
 395.3344421386719,
 581.4437866210938]

In [102]:
display(targets)
display(predictions)

[844.864,
 791.375,
 818.6451428571429,
 676.3001142857142,
 538.5867428571428,
 362.791,
 340.278,
 395.359,
 581.479,
 576.578]

[742.8809814453125,
 793.86962890625,
 777.3408813476562,
 711.07470703125,
 681.372314453125,
 538.5426635742188,
 393.5416564941406,
 367.07562255859375,
 395.3344421386719,
 581.4437866210938]

In [103]:
mae = mean_absolute_error(predictions, targets)
mae

77.16511153041297

In [104]:
porcentage = mae/np.mean(targets)
porcentage

0.13020887307334172